Predictive Thyroid Disease Project - EDA

To make a predictive model where you input factors of a person and have it predict if they are likely to have thyroid disease or not. The accuracy has to be above 95% to be considered successful.

Initial set up

In [1]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder

In [3]:
# load the cleaned data

df = pd.read_csv("Thyroid_Diff_cleaned.csv")

Quick Explore

In [7]:
# Display number of rows, column names, and column data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Age                   383 non-null    int64 
 1   Gender                383 non-null    int64 
 2   Smoking               383 non-null    int64 
 3   Hx_Smoking            383 non-null    int64 
 4   Hx_Radiotherapy       383 non-null    int64 
 5   Thyroid_Function      383 non-null    object
 6   Physical_Examination  383 non-null    object
 7   Adenopathy            383 non-null    object
 8   Pathology             383 non-null    object
 9   Focality              383 non-null    int64 
 10  Risk                  383 non-null    object
 11  T                     383 non-null    object
 12  N                     383 non-null    object
 13  M                     383 non-null    int64 
 14  Stage                 383 non-null    object
 15  Response              383 non-null    ob

In [8]:
# Display the first few rows
df.head()

,Age,Gender,Smoking,Hx_Smoking,Hx_Radiotherapy,Thyroid_Function,Physical_Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurrence
0,27,0,0,0,0,Euthyroid,Single nodular goiter-left,No,Micropapillary,1,Low,T1a,N0,0,I,Indeterminate,0
1,34,0,0,1,0,Euthyroid,Multinodular goiter,No,Micropapillary,1,Low,T1a,N0,0,I,Excellent,0
2,30,0,0,0,0,Euthyroid,Single nodular goiter-right,No,Micropapillary,1,Low,T1a,N0,0,I,Excellent,0
3,62,0,0,0,0,Euthyroid,Single nodular goiter-right,No,Micropapillary,1,Low,T1a,N0,0,I,Excellent,0
4,62,0,0,0,0,Euthyroid,Multinodular goiter,No,Micropapillary,0,Low,T1a,N0,0,I,Excellent,0


In [9]:
# Display the last few rows
df.tail()

,Age,Gender,Smoking,Hx_Smoking,Hx_Radiotherapy,Thyroid_Function,Physical_Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurrence
378,72,1,1,1,1,Euthyroid,Single nodular goiter-right,Right,Papillary,1,High,T4b,N1b,1,IVB,Biochemical Incomplete,1
379,81,1,1,0,1,Euthyroid,Multinodular goiter,Extensive,Papillary,0,High,T4b,N1b,1,IVB,Structural Incomplete,1
380,72,1,1,1,0,Euthyroid,Multinodular goiter,Bilateral,Papillary,0,High,T4b,N1b,1,IVB,Structural Incomplete,1
381,61,1,1,1,1,Clinical Hyperthyroidism,Multinodular goiter,Extensive,Hurthel cell,0,High,T4b,N1b,0,IVA,Structural Incomplete,1
382,67,1,1,0,0,Euthyroid,Multinodular goiter,Bilateral,Papillary,0,High,T4b,N1b,0,IVA,Structural Incomplete,1


In [10]:
# Display random sample

df.sample(n=5, random_state=42)

,Age,Gender,Smoking,Hx_Smoking,Hx_Radiotherapy,Thyroid_Function,Physical_Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurrence
268,32,0,0,0,0,Euthyroid,Single nodular goiter-left,No,Papillary,1,Low,T3a,N0,0,I,Excellent,0
250,30,0,0,0,0,Subclinical Hypothyroidism,Normal,Right,Follicular,1,Low,T3a,N0,0,I,Excellent,0
318,30,0,0,0,0,Euthyroid,Multinodular goiter,Bilateral,Papillary,0,Intermediate,T3a,N1b,0,I,Structural Incomplete,1
331,51,0,0,0,0,Euthyroid,Single nodular goiter-left,No,Follicular,1,Intermediate,T3a,N0,0,I,Structural Incomplete,1
56,43,0,0,1,0,Euthyroid,Single nodular goiter-right,No,Papillary,1,Low,T1b,N0,0,I,Excellent,0
